In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

In [19]:
train = pd.read_csv('toxic_train.csv.zip')
test = pd.read_csv('toxic_test.csv.zip')
test_labels = pd.read_csv('toxic_test_labels.csv.zip')
submission = pd.read_csv('toxic_sample_submission.csv.zip')

In [41]:
train.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
19394,33346e9833575b63,Statement on Senate Passage of Immigration Ref...,0,0,0,0,0,0
11790,1f28836efffe7b3a,""" June 2007 (UTC)\n\n You know what I also fou...",0,0,0,0,0,0
127665,aad027e06369b922,"""\nI'd hate to bring this up again, but this e...",0,0,0,0,0,0
82955,ddf597e49f046122,and hope action is taken on this view,0,0,0,0,0,0
114343,63942d562221da7f,"""\n\nSpeedy deletion of Shree Swami Abhiramdas...",0,0,0,0,0,0


In [42]:
train.shape

(159571, 8)

In [43]:
# Defining model output
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
print(y.shape)
print(y[:10])

(159571, 6)
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 1 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [45]:
# Unbalanced data
toxic_ratio = (y.sum(axis = 1) > 0).sum()/y.shape[0]
print('Share of toxic comments:', toxic_ratio)

Share of toxic comments: 0.10167887648758234


In [48]:
print(train[list_classes].sum())
print()
print(train[list_classes].sum()/y.shape[0])

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64


In [50]:
# Multilabel
for cl in list_classes[1:]:
    N = ((train['toxic'] == 0) & (train[cl] == 1)).sum()
    print(f'Is {cl} but not toxic:', N)

Is severe_toxic but not toxic: 0
Is obscene but not toxic: 523
Is threat but not toxic: 29
Is insult but not toxic: 533
Is identity_hate but not toxic: 103


In [51]:
# Baseline (assuming not toxic)
1-(train[list_classes].sum().values/len(train)).mean()

0.9633412921729722

In [52]:
((y == np.zeros_like(y)).sum(axis=0)/len(y)).mean()

0.9633412921729722

In [53]:
train[list_classes].sum().values/len(train)

array([0.09584448, 0.00999555, 0.05294822, 0.00299553, 0.04936361,
       0.00880486])